# Configuration

In [6]:
config = [
    
# General Processing Parameters
# =============================
    
# Path to trackid partition file.
'--tidfile', "/home/schindlera/experiments/ismir2020_reviews/eval_trackids.csv", 

# Path to directory where intermediate features are stored
"--src", "/home/schindlera/sshfs/spark_master_schindlera/MSD/melspec_128_10seconds_2ch/",

# Path to directory to store merged feature-file
"--dst", "/home/schindlera/sshfs/spark_master_schindlera/MSD/melspec_128_10seconds_2ch/",
    
"--crop",            # Crop longer audio files (comment/uncomment)
"--pad",             # Zero-pad shorter audio files (comment/uncomment)
    
"--length_frames", "880"   # final length of extracted segment (in dimensions / number of frames)
    
]

# Imports

In [7]:
import argparse
import logging
import pandas as pd
import os
import numpy as np
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings('ignore')

Parse Configuration

In [8]:
parser = argparse.ArgumentParser()
parser.add_argument('--tidfile',    type=str)
parser.add_argument('--src',        type=str)
parser.add_argument('--dst',        type=str)
parser.add_argument('--crop', action='store_true')
parser.add_argument('--pad', action='store_true')
parser.add_argument("--length_frames", help="final length of extracted segment (in dimensions / number of frames)", type=int)
parser.add_argument("--log-level",  default=logging.INFO, type=lambda x: getattr(logging, x), help="Configure the logging level.")

if sys.argv[0].find("ipykernel_launcher") != -1:
    args = parser.parse_args(config)
else:
    args = parser.parse_args()

In [9]:
logging.basicConfig(format='%(asctime)s %(levelname)s: %(message)s', level=logging.DEBUG, datefmt='%Y-%m-%d %H:%M:%S')
logging.basicConfig(level=args.log_level)

In [13]:
def merge_melspecs(args):
    
    tids = pd.read_csv(args.tidfile, header=None, index_col=0)

    mel_specs = []
    pbar      = tqdm(total=tids.shape[0])

    i = 0

    for tid in tids.index:

        with np.load(args.src + "/" + tid + ".npz", allow_pickle=True) as npz:

            mel_spec = npz["data"].astype(np.float32)

            if (args.pad) and (mel_spec.shape[1] < args.length_frames):

                zeros = np.zeros((mel_spec.shape[0],args.length_frames,mel_spec.shape[2]), dtype=np.float32)
                zeros[:mel_spec.shape[0], :mel_spec.shape[1], :mel_spec.shape[2]] = mel_spec

                mel_spec = zeros

            if args.crop:
                mel_spec = mel_spec[:,:args.length_frames,:]

            mel_specs.append(mel_spec)

            pbar.update()


    np.savez(args.dst, data=mel_specs, track_ids=tids)

In [ ]:
merge_melspecs(args)

# ISMIR 2020 Paper - Experiments

In [14]:
config = [
'--tidfile', "/home/schindlera/experiments/ismir2020_reviews/eval_partition_trackids_train.csv", 
"--src",     "/home/schindlera/sshfs/spark_master_schindlera/MSD/melspec_128_10seconds_2ch/",
"--dst",     "/home/schindlera/experiments/ismir2020_reviews/melspec_128_10seconds_2ch_train.npz",
"--crop",           
"--pad",            
"--length_frames", "880"  
]

args = parser.parse_args(config)

merge_melspecs(args)

In [15]:
config = [
'--tidfile', "/home/schindlera/experiments/ismir2020_reviews/eval_partition_trackids_val.csv", 
"--src",     "/home/schindlera/sshfs/spark_master_schindlera/MSD/melspec_128_10seconds_2ch/",
"--dst",     "/home/schindlera/experiments/ismir2020_reviews/melspec_128_10seconds_2ch_val.npz",
"--crop",           
"--pad",            
"--length_frames", "880"  
]

args = parser.parse_args(config)

merge_melspecs(args)

In [16]:
config = [
'--tidfile', "/home/schindlera/experiments/ismir2020_reviews/eval_partition_trackids_test.csv", 
"--src",     "/home/schindlera/sshfs/spark_master_schindlera/MSD/melspec_128_10seconds_2ch/",
"--dst",     "/home/schindlera/experiments/ismir2020_reviews/melspec_128_10seconds_2ch_test.npz",
"--crop",           
"--pad",            
"--length_frames", "880"  
]

args = parser.parse_args(config)

merge_melspecs(args)